# Advent of Code 2022

[Website](https://adventofcode.com/2022)

## Day 1: Calorie Counting

Find the Elf carrying the most Calories. How many total Calories is that Elf carrying?

In [ ]:
elf = 0
calories_per_elf = [0]

for line in open('01_input.txt', 'r'):
    if line.strip():
        calories_per_elf[elf] += int(line)
    else:
        elf += 1
        calories_per_elf.append(0)

max(calories_per_elf)

Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [ ]:
calories_per_elf.sort(reverse=True)
sum(calories_per_elf[:3])

## Day 2: Rock Paper Scissors

What would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
def get_shape_score(shape):
    return ord(shape) - 87

def get_result_score(opponents_shape, own_shape):
    diff = ord(own_shape) - ord(opponents_shape)
    if diff == 23:
        return 3
    if diff == 21 or diff == 24:
        return 6
    return 0

score = 0
for line in open('02_input.txt', 'r'):
    opponents_shape, own_shape = line.strip().split()
    score += get_shape_score(own_shape)
    score += get_result_score(opponents_shape, own_shape)
print(score)

Following the Elf's instructions for the second column, what would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
def get_own_shape(opponents_shape, result):
    shape_for_loss = { 'A' : 'Z', 'B' : 'X', 'C' : 'Y' }
    shape_for_draw = { 'A' : 'X', 'B' : 'Y', 'C' : 'Z' }
    shape_for_win = { 'A' : 'Y', 'B' : 'Z', 'C' : 'X' }
    
    match result:
        case 'X':
            return shape_for_loss[opponents_shape]
        case 'Y':
            return shape_for_draw[opponents_shape]
        case _:
            return shape_for_win[opponents_shape]

score = 0
for line in open('02_input.txt', 'r'):
    opponents_shape, result = line.strip().split()
    own_shape = get_own_shape(opponents_shape, result)
    score += get_shape_score(own_shape)
    score += get_result_score(opponents_shape, own_shape)
print(score)

## Day 3: Rucksack Reorganization

Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [ ]:
def to_priority(item):
    if(ord(item) < 91):
        return ord(item) - 38
    return ord(item) - 96

priorities = 0
for line in open('03_input.txt', 'r'):
    compartment_len = len(line)//2
    compartment0, compartment1 = line[:compartment_len], line[compartment_len:]
    duplicate_items = set(compartment0).intersection(compartment1)
    assert len(duplicate_items) == 1
    priorities += to_priority(duplicate_items.pop())
print(priorities)

Find the item type that corresponds to the badges of each three-Elf group. What is the sum of the priorities of those item types?

In [ ]:
lines = open('03_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]
assert len(lines) % 3 == 0

priorities = 0
group_idx = 0
while group_idx < len(lines):
    groups = lines[group_idx:group_idx+3]
    duplicate_items = set(groups[0]).intersection(groups[1]).intersection(groups[2])
    assert len(duplicate_items) == 1
    priorities += to_priority(duplicate_items.pop())
    group_idx += 3
print(priorities)

## Day 4: Camp Cleanup

In how many assignment pairs does one range fully contain the other?

In [ ]:
def get_assignments(line):
    assignment0, assignment1 = line.strip().split(',')
    assignment0 = [int(a) for a in assignment0.split('-')]
    assignment1 = [int(a) for a in assignment1.split('-')]
    return (assignment0, assignment1)

def is_fully_contained(assignments):
    return (assignments[0][0] <= assignments[1][0] and assignments[0][1] >= assignments[1][1]) or \
           (assignments[0][1] <= assignments[1][1] and assignments[0][0] >= assignments[1][0])

count = 0
for line in open('04_input.txt', 'r'):
    assignments = get_assignments(line)
    count += is_fully_contained(assignments)
print(count)

In how many assignment pairs do the ranges overlap?

In [ ]:
def is_overlapping(assignments):
    return (assignments[0][0] <= assignments[1][1] and assignments[0][0] >= assignments[1][0]) or \
           (assignments[0][1] <= assignments[1][1] and assignments[0][1] >= assignments[1][0]) or \
           (assignments[1][0] <= assignments[0][1] and assignments[1][0] >= assignments[0][0]) or \
           (assignments[1][1] <= assignments[0][1] and assignments[1][1] >= assignments[0][0])

count = 0
for line in open('04_input.txt', 'r'):
    assignments = get_assignments(line)
    count += is_overlapping(assignments)
print(count)

## Day 5: Supply Stacks

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [ ]:
import re

def parse_input(lines):
    def simplify(stacks_str):
        result = []
        for line in stacks_str:
            while '  ' in line:
                line = line.replace(']    ', '] [.]')
            result.append(line.replace('] [', '').replace('[', '').replace(']\n', ''))
        return result

    def parse_stacks(stacks_str):
        stacks_str = simplify(stacks_str)

        num_stacks = max(len(line) for line in stacks_str)
        stacks = [[] for _ in range(num_stacks)]

        stacks_str.reverse()
        for line in stacks_str:
            for stack_id in range(len(line)):
                if(line[stack_id] == '.'):
                    continue
                stacks[stack_id].append(line[stack_id])
        return stacks
    
    def parse_operations(operations_str):
        operations = []
        for line in operations_str:
            operations.append({
                'quantity': int(re.search('move (.*) from', line).group(1)),
                'source': int(re.search('from (.*) to', line).group(1)) - 1,
                'target': int(re.search('to (.*)\n', line).group(1)) - 1
                })
        return operations

    legend_line_idx = lines.index(next(filter(lambda line : line.startswith(' 1'), lines)))
    stacks_str = lines[:legend_line_idx]
    operations_str = lines[legend_line_idx+2:]
    return parse_stacks(stacks_str), parse_operations(operations_str)

def process_operation(stacks, operation):
    for _ in range(operation['quantity']):
        crate = stacks[operation['source']].pop()
        stacks[operation['target']].append(crate)
    return stacks

def process(stacks, operations):
    for operation in operations:
        stacks = process_operation(stacks, operation)
    return stacks


lines = open('05_input.txt', 'r').readlines()
stacks, operations = parse_input(lines)
updated_stacks = process(stacks, operations)

for stack in updated_stacks:
    print(stack.pop(), end='')

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [ ]:
def process_operation(stacks, operation):
    crates = [stacks[operation['source']].pop() for _ in range(operation['quantity'])]
    crates.reverse()
    [stacks[operation['target']].append(crate) for crate in crates]
    return stacks

lines = open('05_input.txt', 'r').readlines()
stacks, operations = parse_input(lines)
updated_stacks = process(stacks, operations)

for stack in updated_stacks:
    print(stack.pop(), end='')

## Day 6: Tuning Trouble

How many characters need to be processed before the first start-of-packet marker is detected?

In [ ]:
def is_unique_chars(string):
    return len(set(string)) == len(list(string))

def get_marker_end_position(message, marker_len):
    for start_idx in range(len(message)-marker_len):
        marker = message[start_idx:start_idx+marker_len]
        if(is_unique_chars(marker)):
            return start_idx + marker_len
    raise Exception(f'No marker of length {marker_len} found.')

line = open('06_input.txt', 'r').readline()
get_marker_end_position(line, 4)

How many characters need to be processed before the first start-of-message marker is detected?

In [ ]:
line = open('06_input.txt', 'r').readline()
get_marker_end_position(line, 14)

## Day 7: No Space Left On Device

What is the sum of the total sizes of those directories?

In [ ]:
class Directory:
    subdirs: dict
    "key: name, value: directory"

    files: dict
    "key: name, value: directory"

    total_size: int
    "total size including all subdirs"

    def __init__(self, name, parent = None):
        self.name = name
        self.parent = parent
        self.subdirs = {}
        self.files = {}
        self.total_size = None

def parse_input(lines):
    root = Directory('/')
    current_dir = root

    for line in lines:
        match line.removeprefix('$').split():
            case['dir', name]:
                current_dir.subdirs[name] = Directory(name, current_dir)
            case['cd', target]:
                if target == '/':
                    current_dir = root
                elif target == '..':
                    current_dir = current_dir.parent
                else:
                    current_dir = current_dir.subdirs[target]
            case[size, name] if size.isnumeric():
                current_dir.files[name] = int(size)
    return root

def add_total_dir_sizes(current_dir):
    size = sum(current_dir.files.values())
    for subdir in current_dir.subdirs.values():
        size += add_total_dir_sizes(subdir)
    current_dir.total_size = size
    return size

def calculate_size_sum(current_dir, upper_threshold):
    sum = 0
    if current_dir.total_size < upper_threshold:
        sum += current_dir.total_size
    for subdir in current_dir.subdirs.values():
        sum += calculate_size_sum(subdir, upper_threshold)
    return sum

lines = open('07_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]

root = parse_input(lines)
add_total_dir_sizes(root)
calculate_size_sum(root, 100_000)

What is the total size of that directory?

In [ ]:
from sys import maxsize

free_space = 70_000_000 - root.total_size
space_to_be_freed = 30_000_000 - free_space

def get_cleanup_size(current_dir, space_to_be_freed):
    best_match = maxsize
    if current_dir.total_size >= space_to_be_freed:
        best_match = min(best_match, current_dir.total_size)
    for subdir in current_dir.subdirs.values():
        best_match = min(best_match, get_cleanup_size(subdir, space_to_be_freed))
    return best_match

get_cleanup_size(root, space_to_be_freed)

## Day 8: Treetop Tree House

Consider your map; how many trees are visible from outside the grid?

In [ ]:
import numpy as np

heights = np.genfromtxt('08_input.txt', delimiter=1)

visibilities = np.ones_like(heights)
for idx, height in np.ndenumerate(heights[1:-1,1:-1]):
    row_idx = idx[0] + 1
    col_idx = idx[1] + 1
    max_height_above = np.amax(heights[:row_idx, col_idx])
    max_height_below = np.amax(heights[row_idx+1:, col_idx])
    max_height_right = np.amax(heights[row_idx,:col_idx])
    max_height_left = np.amax(heights[row_idx,col_idx+1:])
    visibilities[row_idx, col_idx] = any(m < height for m in [max_height_above, max_height_below, max_height_right, max_height_left])

np.count_nonzero(visibilities)

What is the highest scenic score possible for any tree?

In [ ]:
def get_viewing_distance(height, neighbour_heights):
    distance = 0
    for neighbour_height in neighbour_heights:
        distance += 1
        if(neighbour_height >= height):
            break
    return distance

scenic_scores = np.ones_like(heights)
for idx, height in np.ndenumerate(heights):
    row_idx = idx[0]
    col_idx = idx[1]
    scenic_scores[idx] *= get_viewing_distance(height, heights[:row_idx, col_idx][::-1]) #look up
    scenic_scores[idx] *= get_viewing_distance(height, heights[row_idx+1:, col_idx]) #look down
    scenic_scores[idx] *= get_viewing_distance(height, heights[row_idx,col_idx+1:]) #look right
    scenic_scores[idx] *= get_viewing_distance(height, heights[row_idx,:col_idx][::-1]) #look left

np.max(scenic_scores)

##  Day 9: Rope Bridge

How many positions does the tail of the rope visit at least once?

In [ ]:
def elementwise_add(a, b):
    return tuple(map(sum, zip(a, b)))

def get_head_move(direction):
    match direction:
        case 'U': return (1, 0)
        case 'D': return (-1, 0)
        case 'R': return (0, 1)
        case 'L': return (0, -1)

def get_tail_move(head, tail):
    def get_diagonal_move(diff):
        if diff < 0: return -1
        if diff > 0: return  1
        return 0

    diff_row = head[0] - tail[0]
    diff_col = head[1] - tail[1]
    if diff_row < -1: return (-1, get_diagonal_move(diff_col))
    if diff_row >  1: return (1, get_diagonal_move(diff_col))
    if diff_col < -1: return (get_diagonal_move(diff_row), -1)
    if diff_col >  1: return (get_diagonal_move(diff_row), 1)
    return (0, 0)

def get_tail_visits(lines):
    head, tail = (0,0), (0,0)
    tail_visits = {(0,0)}

    for line in lines:
        direction, num_steps = line.split()
        head_move = get_head_move(direction)
        for _ in range(int(num_steps)):
            head = elementwise_add(head, head_move)
            tail = elementwise_add(tail, get_tail_move(head, tail))
            tail_visits.add(tail)
    return tail_visits

lines = open('09_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]

len(get_tail_visits(lines))

How many positions does the tail of the rope visit at least once?

In [ ]:
def get_tail_visits(lines, number_of_knots):
    head = (0, 0)
    knots = [(0,0)] * number_of_knots
    tail_visits = {(0,0)}

    for line in lines:
        direction, num_steps = line.split()
        head_move = get_head_move(direction)
        for _ in range(int(num_steps)):
            head = elementwise_add(head, head_move)
            knots[0] = elementwise_add(knots[0], get_tail_move(head, knots[0]))
            for i in range(1, len(knots)):
                knots[i] = elementwise_add(knots[i], get_tail_move(knots[i-1], knots[i]))
            tail_visits.add(knots[-1])
    return tail_visits

len(get_tail_visits(lines, 9))

## Day 10: Cathode-Ray Tube

What is the sum of these six signal strengths?

In [ ]:
lines = open('10_input.txt', 'r').readlines()
commands = [line.strip().split() for line in lines]

def get_stack_history(commands):
    history = [1]
    for command in commands:
        match command:
            case ['addx', summand]:
                history.append(history[-1])
                history.append(history[-1] + int(summand))
            case ['noop']:
                history.append(history[-1])
    return history

history = get_stack_history(commands)
sum(history[i]*(i+1) for i in range(19, 220, 40))

What eight capital letters appear on your CRT?

In [ ]:
#TODO

## Day 11: Monkey in the Middle

What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?

In [ ]:
import math
import re

def get_numbers(string):
    return [int(number) for number in re.findall(r'\d+', string)]

def get_number(string):
    return get_numbers(string)[0]

def get_operation(line):
    if line.endswith('old * old'):
        return lambda a : a * a
    if '+' in line:
        b = get_number(line)
        return lambda a :a + b
    if '*' in line:
        b = get_number(line)
        return lambda a : a * b
    raise Exception(f'Unknown operation in {line}')

class Monkey:
    def __init__(self, description):
        for line in description:
            if line.startswith('Monkey'):
                self.id =  get_number(line)
            elif line.startswith('Starting items'):
                self.items = get_numbers(line)
            elif line.startswith('Operation'):
                self.operation = get_operation(line)
            elif line.startswith('Test'):
                self.test_divisor = get_number(line)
            elif line.startswith('If true'):
                self.receiver_if_true = get_number(line)
            elif line.startswith('If false'):
                self.receiver_if_false = get_number(line)

def get_monkeys(lines):
    descriptions = [lines[i:i+7] for i in range(0, len(lines), 7)]
    return [Monkey(description) for description in descriptions]

def add_relief(item):
    return math.floor(item / 3)

def process_round(monkey, monkeys, num_inspections):
    for item in monkey.items:
        item = monkey.operation(item)
        num_inspections[monkey.id] += 1
        item = add_relief(item)
        if item % monkey.test_divisor == 0:
            monkeys[monkey.receiver_if_true].items.append(item)
        else:
            monkeys[monkey.receiver_if_false].items.append(item)
    monkey.items.clear()

def get_num_inspections(monkeys, num_rounds):
    num_inspections = [0] * len(monkeys)
    for _ in range(num_rounds):
        for monkey in monkeys:
            process_round(monkey, monkeys, num_inspections)
    return num_inspections

def get_level_of_monkey_business(monkeys, num_rounds):
    num_inspections = sorted(get_num_inspections(monkeys, num_rounds))
    return num_inspections[-1] * num_inspections[-2]

lines = open('11_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]

monkeys = get_monkeys(lines)
get_level_of_monkey_business(monkeys, 20)

Starting again from the initial state in your puzzle input, what is the level of monkey business after `10000` rounds?

In [ ]:
monkeys = get_monkeys(lines)

divisor = 1
for monkey in monkeys:
    divisor *= monkey.test_divisor

def add_relief(item):
    return item - (math.floor(item / divisor) * divisor)

get_level_of_monkey_business(monkeys, 10_000)

## Day 12: Hill Climbing Algorithm

What is the fewest steps required to move from your current position to the location that should get the best signal?

In [ ]:
from sys import maxsize

def argmin(dict):
    return min(dict, key=dict.get)

def get_position(lines, marker):
    return get_positions(lines, marker)[0]

def get_positions(lines, marker):
    positions = []
    for row_idx,line in enumerate(lines):
        for col_idx,char in enumerate(line):
            if char == marker:
                positions.append((row_idx, col_idx))
    return positions

def get_height_matrix(lines, start_pos, end_pos):
    height_matrix = [[ord(elevation) for elevation in line] for line in lines]
    height_matrix[start_pos[0]][start_pos[1]] = ord('a')
    height_matrix[end_pos[0]][end_pos[1]] = ord('z')
    return height_matrix

def get_all_pos(heigh_matrix):
    return [(x, y) for x in range(len(height_matrix)) for y in range(len(height_matrix[0]))]

def get_neighbours(pos, height_matrix):
    x, y = pos
    neighbours = []
    if x > 0:
        neighbours.append((x-1, y))
    if x < len(height_matrix)-1:
        neighbours.append((x+1, y))
    if y > 0:
        neighbours.append((x, y-1))
    if y < len(height_matrix[0])-1:
        neighbours.append((x, y+1))
    return neighbours

def is_elevation_passable(current_pos, neighbour_pos):
    current_height = height_matrix[current_pos[0]][current_pos[1]]
    neighbour_height = height_matrix[neighbour_pos[0]][neighbour_pos[1]]
    return neighbour_height <= current_height + 1

def get_shortest_path(height_matrix, start_pos, end_pos):
    dist, prev, unvisited = {}, {}, set()
    for pos in get_all_pos(height_matrix):
        dist[pos] = maxsize
        prev[pos] = None
        unvisited.add(pos)
    dist[start_pos] = 0

    while len(unvisited):
        current = argmin({ pos: dist[pos] for pos in unvisited })
        if current == end_pos:
            break
        unvisited.remove(current)

        neighbours = get_neighbours(current, height_matrix)
        for neighbour in unvisited.intersection(neighbours):
            if not is_elevation_passable(current, neighbour):
                continue
            alternative_dist = dist[current] + 1
            if alternative_dist < dist[neighbour]:
                dist[neighbour] = alternative_dist
                prev[neighbour] = current

    if(prev[end_pos] == None):
        return None
    shortest_path = []
    pos = end_pos
    while pos != None:
        shortest_path.insert(0, pos)
        pos = prev[pos]
    return shortest_path

lines = open('12_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]

start_pos = get_position(lines, 'S')
end_pos = get_position(lines, 'E')
height_matrix = get_height_matrix(lines, start_pos, end_pos)
shortest_path = get_shortest_path(height_matrix, start_pos, end_pos)
len(shortest_path)-1

What is the fewest steps required to move starting from any square with elevation `a` to the location that should get the best signal?

In [ ]:
for i,line in enumerate(lines):
        lines[i] = line.replace('S', 'a')

end_pos = get_position(lines, 'E')

num_steps = maxsize
for start_pos in get_positions(lines, 'a'):
    height_matrix = get_height_matrix(lines, start_pos, end_pos)
    shortest_path = get_shortest_path(height_matrix, start_pos, end_pos)
    if shortest_path and len(shortest_path)-1 < num_steps:
        num_steps = len(shortest_path)-1
num_steps